## This notebook launches the LatissCWFSAlign script, meant for the scriptQueue, but via a notebook.
##### This calculates focus offsets for the hexapod (and also decentering corrections)

In [1]:
import sys
import asyncio
import time
import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, AltAz, ICRS, EarthLocation, Angle
from astroquery.vo_conesearch import ConeSearch
from astroquery.simbad import Simbad

%matplotlib inline
from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control.utils import RotType

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [3]:
script = LatissCWFSAlign(index=1, remotes=True)  # this essentially calls the init method
# make sure all remotes etc are running
await script.start_task

DEBUG:Script.ATCS:atmcs: Adding all resources.
atmcs: Adding all resources.
DEBUG:Script.ATCS:atptg: Adding all resources.
atptg: Adding all resources.
DEBUG:Script.ATCS:ataos: Adding all resources.
ataos: Adding all resources.
DEBUG:Script.ATCS:atpneumatics: Adding all resources.
atpneumatics: Adding all resources.
DEBUG:Script.ATCS:athexapod: Adding all resources.
athexapod: Adding all resources.
DEBUG:Script.ATCS:atdome: Adding all resources.
atdome: Adding all resources.
DEBUG:Script.ATCS:atdometrajectory: Adding all resources.
atdometrajectory: Adding all resources.
DEBUG:Script.LATISS:atcamera: Adding all resources.
atcamera: Adding all resources.
DEBUG:Script.LATISS:atspectrograph: Adding all resources.
atspectrograph: Adding all resources.
DEBUG:Script.LATISS:atheaderservice: Adding all resources.
atheaderservice: Adding all resources.
DEBUG:Script.LATISS:atarchiver: Adding all resources.
atarchiver: Adding all resources.
INFO:Script:latiss_cwfs_align initialized!
latiss_cwfs_a

In [4]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

currentTargetStatus DDS read queue is full (100 elements); data may be lost
mount_Nasmyth_Encoders DDS read queue is filling: 38 of 100 elements
mount_AzEl_Encoders DDS read queue is filling: 38 of 100 elements


In [5]:
customSimbad = Simbad()
customSimbad.add_votable_fields('distance_result', "fluxdata(V)")
customSimbad.TIMEOUT = 30

measuredTorque DDS read queue is filling: 38 of 100 elements
measuredMotorVelocity DDS read queue is filling: 38 of 100 elements
mount_AzEl_Encoders python read queue is filling: 37 of 100 elements
azEl_mountMotorEncoders DDS read queue is filling: 38 of 100 elements
target DDS read queue is filling: 13 of 100 elements
target python read queue is filling: 12 of 100 elements


In [13]:
def find_target(az, el, mag_limit):
    current_time = Time.now()

    elaz = AltAz(alt=el*u.deg, az=az*u.deg, 
                 obstime=current_time.tai, 
                 location=location)

    radec = elaz.transform_to(ICRS)

    _ra = radec.ra.to_string(u.deg, decimal=True)
    _dec = radec.dec.to_string(u.deg, decimal=True, alwayssign=True)
    r = Angle(2.*u.deg).to_string(u.deg, decimal=True)

    criteria = f"region(circle,ICRS,{_ra} {_dec},{r}d) & Vmag > {mag_limit} & Vmag < {mag_limit+2} & cat = HD"

    try:
        result_table = customSimbad.query_criteria(criteria)
        if result_table is None:
            raise RuntimeError("No result from query.")
    except Exception as e:
        print(f"Error getting target from Simbad. Skipping target {elaz!r}.")
        print(criteria)
        return -1

    result_table.sort('FLUX_V')

    target_radec = SkyCoord(Angle(result_table['RA'][0], unit=u.hour), 
                    Angle(result_table['DEC'][0], unit=u.deg), frame=ICRS)

    t_id = str(result_table['MAIN_ID'][0])
    t_ra = result_table['RA'][0]
    t_dec = result_table['DEC'][0]
    t_mag = result_table['FLUX_V'][0]
    print(f">>>> Point to {t_id:15}: {t_ra} {t_dec} (VMag:{t_mag:5.2f})")

    return 0

In [147]:
find_target(180, 30, 8.0)

DEBUG:urllib3.connectionpool:Resetting dropped connection: simbad.u-strasbg.fr
Resetting dropped connection: simbad.u-strasbg.fr
DEBUG:urllib3.connectionpool:http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
>>>> Point to HD  83745      : 08 57 15.9859 -88 16 14.390 (VMag: 8.01)


0

target python read queue is filling: 48 of 100 elements


In [148]:
# Slew to an object if not already there.
await script.atcs.slew_object('HD  83745 ', rot_type=RotType.PhysicalSky)

DEBUG:urllib3.connectionpool:Resetting dropped connection: simbad.u-strasbg.fr
Resetting dropped connection: simbad.u-strasbg.fr
DEBUG:urllib3.connectionpool:http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
INFO:Script.ATCS:Slewing to HD  83745 : 08 57 15.9859 -88 16 14.390
Slewing to HD  83745 : 08 57 15.9859 -88 16 14.390
DEBUG:Script.ATCS:Setting rotator physical position to 0.0 deg. Rotator will track sky.
Setting rotator physical position to 0.0 deg. Rotator will track sky.
DEBUG:Script.ATCS:Sending command
Sending command
DEBUG:Script.ATCS:Stop tracking.
Stop tracking.
target python read queue is filling: 22 of 100 elements
DEBUG:Script.ATCS:Tracking state: <AtMountState.TRACKINGENABLED: 9>
Tracking state: <AtMountState.TRACKINGENABLED: 9>
DEBUG:Script.ATCS:Tracking state: <AtMountState.STOPPING: 10>
Tracking state: <AtMountState.STOPPING: 10>
DEBUG:Script.ATCS:In Position: True.
In

In [149]:
await script.latiss.take_object(5.0, 1)

DEBUG:Script.LATISS:Generating group_id
Generating group_id
DEBUG:Script.LATISS:imagetype: OBJECT, wait for TCS to be ready.
imagetype: OBJECT, wait for TCS to be ready.
DEBUG:Script.ATCS:atspectrograph correction running. Trying to determine state of the corrections.
atspectrograph correction running. Trying to determine state of the corrections.
DEBUG:Script.ATCS:No correction seen in the last 5.0 seconds. Determining order of last corrections.
No correction seen in the last 5.0 seconds. Determining order of last corrections.
DEBUG:Script.ATCS:Last correction completed.
Last correction completed.
DEBUG:Script.ATCS:Ready to take data:: atmcs=True, ataos=True, atdome=True.
Ready to take data:: atmcs=True, ataos=True, atdome=True.
DEBUG:Script.LATISS:OBJECT 0001 - 0001
OBJECT 0001 - 0001


array([2021070700462])

## Emulate how the scriptQueue launches scripts
##### Start here if re-running the script after a correction

In [17]:
configuration = yaml.safe_dump({"filter": 'RG610', 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook'})
print(configuration)

dataPath: /project/shared/auxTel/rerun/quickLook
exposure_time: 20
filter: RG610
grating: empty_1



In [18]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

INFO:Script:Using binning factor of 1
Using binning factor of 1


In [160]:
# This will run the cwfs multiple times until it converges,
# then apply the calculated offsets
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10
results = await script.arun()

DEBUG:Script:CWFS iteration 1 starting...
CWFS iteration 1 starting...
Intra/Extra images not taken. Running take image sequence.
DEBUG:Script:Moving to intra-focal position
Moving to intra-focal position
DEBUG:Script.LATISS:imagetype: ENGTEST, skip TCS synchronization.
imagetype: ENGTEST, skip TCS synchronization.
DEBUG:Script.LATISS:ENGTEST 0001 - 0001
ENGTEST 0001 - 0001
DEBUG:Script:Moving to extra-focal position
Moving to extra-focal position
DEBUG:Script:Taking extra-focal image
Taking extra-focal image
DEBUG:Script.LATISS:imagetype: ENGTEST, skip TCS synchronization.
imagetype: ENGTEST, skip TCS synchronization.
DEBUG:Script.LATISS:ENGTEST 0001 - 0001
ENGTEST 0001 - 0001
INFO:Script:intraImage expId for target: 2021070700502
intraImage expId for target: 2021070700502
INFO:Script:extraImage expId for target: 2021070700503
extraImage expId for target: 2021070700503
INFO:Script:angle used in cwfs algorithm is -38.70
angle used in cwfs algorithm is -38.70
DEBUG:Script:Moving hexapod

In [ ]:
focus_offset_summary = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()

In [ ]:
print(focus_offset_summary)

In [ ]:
# This will just run the CWFS one time
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10
results = await script.run_cwfs()

In [ ]:
# show donuts and centroids
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script.intra_visit_id}")
ax11.imshow(script.I1[0].image0)
ax11.contour(script.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script.extra_visit_id}")
ax12.imshow(script.I2[0].image0)
ax12.contour(script.algo.pMask) 

In [ ]:
# Apply calculated focus offset
calculated_hexapod_focus_offset = results['hex_offset'][2]
print(f'Applying focus offset of {calculated_hexapod_focus_offset}')
await script.atcs.rem.ataos.cmd_offset.set_start(z=calculated_hexapod_focus_offset)

In [ ]:
await script.atcs.slew_object('HD185975', rot_type=2)

In [ ]:
await script.latiss.take_object(5.0)

# Stop here unless a re-reduction of the doughnuts is required.

#### If you want to re-reduce data then use the below cells

In [ ]:
# Show which files/parameters were taken in the sequence above
print(f'intra_visit_id is {script.intra_visit_id}')
print(f'extra_visit_id is {script.extra_visit_id}')
print(f'angle is {script.angle}')

In [ ]:
# If desired then different filenames can be manually input here
#script.intra_visit_id=2021011900169 
#script.extra_visit_id=2021011900170 
#script.angle=-91.56748047249727

In [ ]:
# reruns reduction part only. ALL 3 fields above must be set! 
rerun_results = await script.run_cwfs()